In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2


In /home/hoangdzung/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/hoangdzung/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/hoangdzung/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/hoangdzung/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/hoangdzung/.local/lib/python3.6/site-packages/m

In [17]:
from crossval import GridSearchCV
from implementations import logistic_regression, reg_logistic_regression
from proj1_helpers import load_csv_data, predict_labels, acc_score, f1_score
from preprocessing import NonLinearTransformer, Normalizer, Imputer, PolynomialFeature, Pipeline, remove_outliers

In [13]:
DATA_TRAIN_PATH = 'data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
DATA_TEST_PATH = './data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [20]:
gridsearch = GridSearchCV(logistic_regression, predict_labels, [f1_score, acc_score], {'gamma':[0.01,0.1,1]} )

In [21]:
pipeline = Pipeline(Imputer(dropnan=True), Normalizer())

In [22]:
gridsearch.fit(y,tX, pipeline, addition_on_train=remove_outliers)

KeyboardInterrupt: 